# 文本预处理

In [13]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

In [14]:
# 定义 'time_machine' 数据集的 URL 和校验和
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 清洗并标准化每行的文本内容，去除非字母字符并转换为小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

# 读取时间机器数据
lines = read_time_machine()

print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

In [15]:
def tokenize(lines, token='word'):
    # 如果标记类型是 'word'，将每一行分割成单词列表。
    if token == 'word':
        return [line.split() for line in lines]
    # 如果标记类型是 'char'，将每一行分割成字符列表。
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)

for i in range(11):
    print(tokens[i])  # 打印第i行的词元

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表

In [16]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率对tokens进行排序
        counter = count_corpus(tokens)
        # 根据counter生成键值对,根据对应键出现的次数进行降序排序(reverse=True)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens  # 索引0处为'<unk>'未知词元，其后是保留词元
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}  # 创建词元到索引的映射关系
        for token, freq in self._token_freqs:  # 遍历按出现频率排序后的词元及其频率
            if freq < min_freq:  # 如果频率小于最小频率，则跳出循环
                break
            if token not in self.token_to_idx:  # 如果词元不在映射中，则添加到映射中
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):  # 返回词汇表中词元的数量
        return len(self.idx_to_token)

    def __getitem__(self, tokens):  # 获取词元对应的索引
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)  # 返回词元对应的索引，如果不存在则返回未知词元索引
        return [self.__getitem__(token) for token in tokens]  # 否则遍历tokens中的每个词元并获取其索引

    def to_tokens(self, indices):  # 根据索引返回词元
        if not isinstance(indices, (list, tuple)):  # 如果indices不是列表或元组
            return self.idx_to_token[indices]  # 返回对应索引处的词元
        return [self.idx_to_token[index] for index in indices]  # 否则遍历indices中的每个索引并获取对应的词元

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):  # 返回词元频率列表
        return self._token_freqs

def count_corpus(tokens):
    # 词元出现次数的统计结果
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]  # 将tokens扁平化为一维列表
    return collections.Counter(tokens)  # 返回统计结果


In [17]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])
print(list(vocab.idx_to_token)[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]
['<unk>', 'the', 'i', 'and', 'of', 'a', 'to', 'was', 'in', 'that']


In [18]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 整合所有功能

In [19]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)